
[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: C:\Users\91993\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [31]:
import numpy as np
import pandas as pd

In [32]:
import os
for dirname, _, filenames in os.walk('./Data/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

./Data/gender_submission.csv
./Data/test.csv
./Data/train.csv


In [33]:
DATA_DIR = './Data/'
TRAIN_DATA = 'train.csv'
TEST_DATA = 'test.csv'

In [34]:
train_data = pd.read_csv(DATA_DIR + TRAIN_DATA)
test_data = pd.read_csv(DATA_DIR + TEST_DATA)
test_ids = test_data["PassengerId"]
train_data.head(2)
test_data.head(2)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S


In [35]:
def clean(data):
    data = data.drop(["Ticket", "Name", "Cabin", "PassengerId"], axis=1)
    cols = ["SibSp", "Parch", "Fare", "Age"]
    for col in cols:
        data[col].fillna(data[col].median(), inplace=True)
    data.Embarked.fillna("U", inplace=True)
    return data

train_data = clean(train_data)
test_data = clean(test_data)

C:\Users\91993\AppData\Local\Temp\ipykernel_27152\1601930289.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data[col].fillna(data[col].median(), inplace=True)
C:\Users\91993\AppData\Local\Temp\ipykernel_27152\1601930289.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For e

In [36]:
train_data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [37]:
from sklearn import preprocessing

In [38]:
le = preprocessing.LabelEncoder()
cols = ["Sex", "Embarked"]

for col in cols:
    train_data[col] = le.fit_transform(train_data[col])
    test_data[col] = le.fit_transform(test_data[col])
    print (le.classes_)

train_data.head() 

['female' 'male']
['C' 'Q' 'S']


,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22.0,1,0,7.2500,2
1,1,1,0,38.0,1,0,71.2833,0
2,1,3,0,26.0,0,0,7.9250,2
3,1,1,0,35.0,1,0,53.1000,2
4,0,3,1,35.0,0,0,8.0500,2


In [39]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

y = train_data["Survived"]
x = train_data.drop(["Survived"], axis=1)

X_train, X_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=42)

In [40]:
clf = LogisticRegression(random_state=0, max_iter=1000).fit(X_train, y_train) 

In [41]:
predictions = clf.predict(X_val)
from sklearn.metrics import accuracy_score
accuracy_score(y_val, predictions)

0.8100558659217877

In [42]:
submission_predictions = clf.predict(test_data)

In [43]:
df = pd.DataFrame({
    "PassengerId": test_ids,
    "Survived": submission_predictions
})

In [46]:
df.to_csv('submission.csv', index=False)